In [ ]:
# This notebook has two cells that support the consumption and testing of the new Managed Endpoint

# the new user is required to update/edit both cells to provided: the Endpoint URL, API Primary key, and headers string, 
# to match the names and specs of the user's inferencing model and new Endpoint

# the first cell constructs a single HTTP REST POST Request using a remotely-stored image URL and sends it too the Endpoint who inferences it, 
# and returns a JSON dictionary with the bounding box(es), probability(s), and category(s) of any objects their model identifies within the image.  

# the second cell repeats these same steps, but wraps them inside a simple python looping action to repeat them in volume and provide a modest load test for the new Endpoint


# both cells output to the terminal window/log, and return a parseable JSON dict to the requesting service

# in a future improvement - it would be a good idea to pass in Probability Threshold in as a header key:value pair... so at run time the app developer can alter this...


In [1]:
# 2024Feb18 Later update for publication, new rg and new AML resource, new AML Workspace, new Notebook repo for demo Endpoint 021cw hosting CWD OD model.

import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script

# image_url = "https://asatestimages01.blob.core.windows.net/container-testimages/DEER_CWD_imagery/Cropped_IC_Model/DeerDayHealthyCandidates/TestImageBlob.jpg" 

# or choose one from the image test files below

# image_url = "https://asatestimages01.blob.core.windows.net/container-testimages/DEER_CWD_imagery/Mixed_Deer_Images_for_Testing/UnHealthySSWI000000012444874A.jpg" 
# image_url = "https://asatestimages01.blob.core.windows.net/container-testimages/DEER_CWD_imagery/Mixed_Deer_Images_for_Testing/UnHealthySSWI000000012733278A.jpg" 
# image_url = "https://asatestimages01.blob.core.windows.net/container-testimages/DEER_CWD_imagery/Mixed_Deer_Images_for_Testing/UnHealthySSWI000000012931348B.jpg" 
image_url = "https://asatestimages01.blob.core.windows.net/container-testimages/DEER_CWD_imagery/Mixed_Deer_Images_for_Testing/UnHealthySSWI000000013042672A.jpg" 
# image_url = "https://asatestimages01.blob.core.windows.net/container-testimages/DEER_CWD_imagery/Mixed_Deer_Images_for_Testing/UnHealthySSWI000000013486750A.jpg" 
# image_url = "https://asatestimages01.blob.core.windows.net/container-testimages/DEER_CWD_imagery/Mixed_Deer_Images_for_Testing/HealthySSWI000000016989655A.jpg" 
# image_url = "https://asatestimages01.blob.core.windows.net/container-testimages/DEER_CWD_imagery/Mixed_Deer_Images_for_Testing/HealthySSWI000000017061751A.jpg" 
# image_url = "https://asatestimages01.blob.core.windows.net/container-testimages/DEER_CWD_imagery/Mixed_Deer_Images_for_Testing/HealthySSWI000000017167085C.jpg" 
# image_url = "https://asatestimages01.blob.core.windows.net/container-testimages/DEER_CWD_imagery/Mixed_Deer_Images_for_Testing/HealthySSWI000000016685494B.jpg" 
# image_url = "https://asatestimages01.blob.core.windows.net/container-testimages/DEER_CWD_imagery/Mixed_Deer_Images_for_Testing/HealthySSWI000000016182548A.jpg" 

# Label Data Legend
# 0 deer-day-healthy      # if model returns 0 --> HEALTHY Deer,     1 --> UNHealthy Deer
# 1 deer-day-unhealthy
# above labels are from labels.txt, Note - Line numbers 1 and 2 map to read-in Index numbers 0 and 1


urlData = {
    "image_url": image_url
}

body = str.encode(json.dumps(urlData))


#-----------------------------------------
# this section contains the params the user needs to update for any new Endpoint hosting this Chronic Wasting Disease (CWD) .ONNX model

url = 'https://aml-aivision-02-021cw.northcentralus.inference.ml.azure.com/score'  # NAR AML demo resource

# Replace this with the primary/secondary key or AMLToken for the endpoint
api_key = 'RhVWhz4WsNzADQJYdWQLTpE00vJWtkJh'  # NAR demo resource api key
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'cwdodonnxmodelfromotherframew-1' , 'detThresh': '0.20'} # NAR Deployment demo
#-----------------------------------------

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)
    print("response is type: ", type(response))

    result = response.read()
    print("result is type: ", type(result))

    dictResult = json.loads(json.loads(result)) # yes, this needed to be nested twice...
    print("dictResult is type: ", type(dictResult))
    print("dictResult = ", dictResult)
    # at this stage, dictResult contained a valid correct json dictionary - ready to go into other code...

except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))


response is type:  <class 'http.client.HTTPResponse'>
result is type:  <class 'bytes'>
dictResult is type:  <class 'dict'>
dictResult =  {'detected_boxes': [0.5344937443733215, 0.6151592135429382, 0.7772806286811829, 0.9887133240699768, 0.0015653371810913086, 0.4758493900299072, 0.656927227973938, 0.9640614986419678, -0.00420689582824707, 0.08173990249633789, 0.902786374092102, 0.8089191913604736, 0.03439095616340637, 0.135395348072052, 0.5588785409927368, 0.7592920660972595], 'detected_classes': [1, 0, 0, 1], 'detected_scores': [0.9815239906311035, 0.14228025078773499, 0.08871999382972717, 0.011512607336044312]}


In [ ]:
# 2024Feb18 Later update for publication, new rg and new AML resource, new AML Workspace, new Notebook repo for demo Endpoint 021cw hosting CWD OD model.

import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script

# image_url = "https://asatestimages01.blob.core.windows.net/container-testimages/DEER_CWD_imagery/Cropped_IC_Model/DeerDayHealthyCandidates/TestImageBlob.jpg" # NAR demo image

# or choose one from the image test files below

# image_url = "https://asatestimages01.blob.core.windows.net/container-testimages/DEER_CWD_imagery/Mixed_Deer_Images_for_Testing/UnHealthySSWI000000012444874A.jpg" 
# image_url = "https://asatestimages01.blob.core.windows.net/container-testimages/DEER_CWD_imagery/Mixed_Deer_Images_for_Testing/UnHealthySSWI000000012733278A.jpg" 
# image_url = "https://asatestimages01.blob.core.windows.net/container-testimages/DEER_CWD_imagery/Mixed_Deer_Images_for_Testing/UnHealthySSWI000000012931348B.jpg" 
# image_url = "https://asatestimages01.blob.core.windows.net/container-testimages/DEER_CWD_imagery/Mixed_Deer_Images_for_Testing/UnHealthySSWI000000013042672A.jpg" 
# image_url = "https://asatestimages01.blob.core.windows.net/container-testimages/DEER_CWD_imagery/Mixed_Deer_Images_for_Testing/UnHealthySSWI000000013486750A.jpg" 
# image_url = "https://asatestimages01.blob.core.windows.net/container-testimages/DEER_CWD_imagery/Mixed_Deer_Images_for_Testing/HealthySSWI000000016989655A.jpg" 
image_url = "https://asatestimages01.blob.core.windows.net/container-testimages/DEER_CWD_imagery/Mixed_Deer_Images_for_Testing/HealthySSWI000000017061751A.jpg" 
# image_url = "https://asatestimages01.blob.core.windows.net/container-testimages/DEER_CWD_imagery/Mixed_Deer_Images_for_Testing/HealthySSWI000000017167085C.jpg" 
# image_url = "https://asatestimages01.blob.core.windows.net/container-testimages/DEER_CWD_imagery/Mixed_Deer_Images_for_Testing/HealthySSWI000000016685494B.jpg" 
# image_url = "https://asatestimages01.blob.core.windows.net/container-testimages/DEER_CWD_imagery/Mixed_Deer_Images_for_Testing/HealthySSWI000000016182548A.jpg" 

# Label Data Legend
# 0 deer-day-healthy      # if model returns 0 --> HEALTHY Deer,     1 --> UNHealthy Deer
# 1 deer-day-unhealthy
# above labels are from labels.txt, Note - Line numbers 1 and 2 map to read-in Index numbers 0 and 1

urlData = {
    "image_url": image_url
}

body = str.encode(json.dumps(urlData))

#-----------------------------------------------------------------------------------------------------------------------
# this section contains the params the user needs to update for any new Endpoint hosting this Chronic Wasting Disease (CWD) .ONNX model

url = 'https://aml-aivision-02-021cw.northcentralus.inference.ml.azure.com/score'  # NAR AML demo resource

# Replace this with the primary/secondary key or AMLToken for the endpoint
api_key = 'RhVWhz4WsNzADQJYdWQLTpE00vJWtkJh'  # NAR demo resource api key
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'cwdodonnxmodelfromotherframew-1' , 'detThresh': '0.20'} # NAR Deployment demo
#------------------------------------------------------------------------------------------------------------------------

req = urllib.request.Request(url, body, headers)

# -------- everything ABOVE this line is setting up the POST request -------------

for i in range(2500):

    try:
        response = urllib.request.urlopen(req)
        print("response is type: ", type(response))

        result = response.read()
        print("result is type: ", type(result))

        dictResult = json.loads(json.loads(result)) # yes, this needed to be nested twice...
        print("dictResult is type: ", type(dictResult))
        print("dictResult = ", dictResult)
        # at this stage, dictResult contained a valid correct json dictionary - ready to go into other code...
        print("iteration i = ", i)

    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))

        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))
